In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
import matplotlib.pyplot as plt
import xgboost as xgb
import pandas as pd

In [ ]:
# df = pd.read_csv("/cephfs/projects/ikozmin/TEST/scoring_case.csv")
df = pd.read_csv("../data/scoring_case.csv")

for column in df.columns:
    if str(df[column].dtype) == 'object':
        df[column] = pd.factorize(df[column])[0]

In [ ]:
df.dropna(axis=1, thresh=int(0.8 * df.shape[0]), inplace=True)
df = df.drop(["FLAG_MOBIL", "SK_ID_CURR"], axis=1)
# 1 - df.isna().sum() / len(df)

In [ ]:
for column in df.columns:
    if column == 'TARGET':
        continue

    df[column] = df[column].fillna(round(df[column].mode()[0]))

In [ ]:
df = df.dropna()

In [ ]:
result = []

In [ ]:
x = df.drop(["TARGET"], axis=1)
y = df["TARGET"]

random_state = 64
# Разделите данные на обучающий и тестовый наборы
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=random_state)

# Подсчитайте количество примеров в каждом классе
count_class_0, count_class_1 = y_train.value_counts()

scale_pos_weight = count_class_0 / count_class_1

param_grid = {
    'max_depth': range(1, 20),
    'n_estimators': range(10, 1000, 20),
    'learning_rate': [0.001, 0.005, 0.01, 0.05, 0.1, 0.2],
    'eta': [x/10 for x in range(10)],
    'gamma': [x/10 for x in range(10)],
    'min_child_weight': [1, 5, 10, 15, 30, 100],
    'max_delta_step': range(10),
}

model = xgb.XGBClassifier(random_state=random_state,
                          scale_pos_weight=scale_pos_weight,
                          objective='binary:logistic')

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

for i in range(2):
    grid_search = RandomizedSearchCV(estimator=model,
                                     param_distributions=param_grid,
                                     scoring='roc_auc',
                                     cv=cv,
                                     n_iter=1,
                                     verbose=1,
                                     random_state=i)
    grid_search.fit(x_train, y_train)
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    
    # Обучение
    best_model = xgb.XGBClassifier(random_state=random_state, scale_pos_weight=scale_pos_weight, **best_params)
    best_model.fit(x_train, y_train)
    
    result.append([roc_auc_score(y_train, best_model.predict(x_train)),
                   roc_auc_score(y_test, best_model.predict(x_test)),
                   best_params])
    print(i)
sorted(result, key=lambda g: g[1])[-1]

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5))
title = ["train", "test"]
for i, data in enumerate([[x_train, y_train], [x_test, y_test]]):
    prediction, target_list = best_model.predict(data[0]), data[1]
    logit_roc_aut = roc_auc_score(target_list, prediction)
    fpr, tpr, thresholds = roc_curve(target_list, prediction)

    axs[i].plot(fpr, tpr, logit_roc_aut)
    axs[i].plot([0, 1], [0, 1], '--')
    axs[i].set_title(f"{title[i]} - ({round(logit_roc_aut, 4)})")
plt.show()